In [45]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
import pprint

In [46]:
#import os
#os.environ["OPENAI_API_KEY"]="YOUR_OPENAI_API_KEY"

In [47]:
class CountryInfo(BaseModel):
    country: str = Field(..., description="The name of the country")
    capital: str = Field(..., description="The capital city of the country")

In [48]:
output_parser = PydanticOutputParser(pydantic_object=CountryInfo)
output_parser

PydanticOutputParser(pydantic_object=<class '__main__.CountryInfo'>)

In [59]:
text_template = """
Give me only one country that speak {language} and its capitals.
Format instructions: 
{format_instructions}
"""


In [60]:

prompt = PromptTemplate(
    input_variables=["language"],
    template=text_template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)
prompt

PromptTemplate(input_variables=['language'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"country": {"description": "The name of the country", "title": "Country", "type": "string"}, "capital": {"description": "The capital city of the country", "title": "Capital", "type": "string"}}, "required": ["country", "capital"]}\n```'}, template='\nGive me only one country that speak {language} and its capitals.\nFormat instructions: \n{format_instructions}\n')

In [61]:
final_prompt = prompt.format_prompt(language="quechua")
print(final_prompt.to_string())


Give me only one country that speak quechua and its capitals.
Format instructions: 
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"country": {"description": "The name of the country", "title": "Country", "type": "string"}, "capital": {"description": "The capital city of the country", "title": "Capital", "type": "string"}}, "required": ["country", "capital"]}
```



In [62]:
llm = OpenAI(temperature=1)


In [63]:
output = llm.invoke(input=final_prompt.to_string())

In [64]:
output

'Here is the code providing the country and its capital:\n\n```\n\n{\n    "country": "Peru",\n    "capital": "Lima"\n}'

In [65]:
type(output)

str

In [67]:
elements = output_parser.parse(output)
elements

CountryInfo(country='Peru', capital='Lima')